In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances, f1_score, make_scorer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import GroupKFold, cross_val_score, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

Извлекаем ифнформацию о заголовках из файла 'docs_titles.tsv'. Рекомендуется использовать его только для домашних работ, но строгого запрета нет. Если что, можно пересобрать руками из content.tar.gz

In [2]:
#Блок кода из project_overview
doc_to_title = {}
with open('docs_titles.tsv', encoding='utf-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

28026


In [3]:
doc_to_title

{15731: 'ВАЗ 21213 | Замена подшипников ступицы | Нива',
 14829: 'Ваз 2107 оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
 15764: 'Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена, замена, тюнинг.',
 17669: 'Классика 21010 - 21074',
 14852: 'Ступица Нива — замена подшипника своими руками',
 15458: 'ВАЗ 2110',
 14899: 'Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123',
 16879: 'Купить Подшипники и ступицы FAG (Страница 23)',
 16310: 'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года',
 15440: 'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти',
 16242: 'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
 16383: 'Ступицы - OLX.ua - страница 80',
 15580: 'маааленькая проблемка — бортжурнал Автокам 2160 ╬ 1994 года на DRIVE2',
 16840: 'Разгруженные полуоси для Нива (24 шлица 765 мм)',
 17519: 'Прошивки для нива м7.9.7 скачать - Фа

Разбиваем в словарь по группам. Ключ - номер группы.

In [4]:
#Блок кода из project_overview
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [5]:
doc_to_title[0] = ''

In [8]:
doc_to_title

{15731: 'ВАЗ 21213 | Замена подшипников ступицы | Нива',
 14829: 'Ваз 2107 оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
 15764: 'Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена, замена, тюнинг.',
 17669: 'Классика 21010 - 21074',
 14852: 'Ступица Нива — замена подшипника своими руками',
 15458: 'ВАЗ 2110',
 14899: 'Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123',
 16879: 'Купить Подшипники и ступицы FAG (Страница 23)',
 16310: 'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года',
 15440: 'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти',
 16242: 'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
 16383: 'Ступицы - OLX.ua - страница 80',
 15580: 'маааленькая проблемка — бортжурнал Автокам 2160 ╬ 1994 года на DRIVE2',
 16840: 'Разгруженные полуоси для Нива (24 шлица 765 мм)',
 17519: 'Прошивки для нива м7.9.7 скачать - Фа

Токенизируем текст

In [6]:
#Имеет смысл доработать и применить nltk, pymorphy
doc_to_title_tokenized = dict()
for key in doc_to_title:
    sentence_token = doc_to_title[key]
    sentence_token = sentence_token.lower().split()
    doc_to_title_tokenized[key] = sentence_token

In [7]:
doc_to_title_tokenized

{15731: ['ваз', '21213', '|', 'замена', 'подшипников', 'ступицы', '|', 'нива'],
 14829: ['ваз',
  '2107',
  'оптом',
  'в',
  'сочи.',
  'сравнить',
  'цены,',
  'купить',
  'потребительские',
  'товары',
  'на',
  'tiu.ru'],
 15764: ['купить',
  'ступица',
  'лада',
  'калина2.',
  'трансмиссия',
  '-',
  'переходные',
  'ступицы',
  'цена,',
  'замена,',
  'тюнинг.'],
 17669: ['классика', '21010', '-', '21074'],
 14852: ['ступица', 'нива', '—', 'замена', 'подшипника', 'своими', 'руками'],
 15458: ['ваз', '2110'],
 14899: ['обзор', 'подшипников', 'полуоси', 'ваз', '2101-07,', '2121,2123'],
 16879: ['купить', 'подшипники', 'и', 'ступицы', 'fag', '(страница', '23)'],
 16310: ['horsepowers',
  '—',
  'автомобильный',
  'интернет',
  'портал',
  '»',
  'отзыв',
  'владельца',
  'ваз',
  '2121',
  'нива',
  '2007',
  'года'],
 15440: ['новости',
  'и',
  'сообщения',
  'из',
  'официальной',
  'группы',
  'вконтакте',
  'торговой',
  'компании',
  '33',
  'sport',
  '-',
  'магазины',
  '-

Более тщательный подбор параметров векторайзера

In [7]:
vectorizer = TfidfVectorizer()
prep_vec = []
for key in range(0, len(doc_to_title_tokenized)):
    temp = ' '.join(doc_to_title_tokenized[key])
    prep_vec.append(temp)
data_vec = vectorizer.fit_transform(prep_vec)
data_vec.shape

(28027, 41736)

Для каждого дока в группе - 15 косинусных расстояний до ближайших в качестве признаков

In [11]:
def prepare_cosine(filename, arg = 1):
    df = pd.read_csv(filename)
    X1 = np.zeros((df.shape[0], 15), dtype=float)
    if arg == 1:
        y1 = np.zeros(df.shape[0], dtype=float)
    used = 0
    for num in df.group_id.unique():
        temp = df[df.group_id == num]
        temp_docs = temp.doc_id
        dist_mat = pairwise_distances(data_vec[temp.doc_id], metric='cosine')
        for i in range(0, dist_mat.shape[0]):
            cur_dist = np.sort(dist_mat[i])[1:16]
            X1[used + i] = cur_dist
        if arg == 1:
            y1[used:temp.shape[0] + used] = temp['target']
        used += temp.shape[0]
    if arg == 1:
        return X1, y1
    else:
        return X1

Есть смысл попробовать большее число косинусный расстояний, возможно какие-то их отлонения и средние. Есть смысл выкачать urlы, и посчитать расстояния для них. Что-то сделать с ключевыми словами и другой разметкой. Разметить кластеризациями по расстояниям. Возможно knn.

In [15]:
X_train, y_train = prepare_cosine('train_groups.csv')

In [18]:
X_test = prepare_cosine('test_groups.csv', arg= 0)

In [23]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
df = pd.read_csv('train_groups.csv')
train_data.group_id.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129],
      dtype=int64)

Есть смысл в более точной кросс-валидации, выяснении смысла других параметров, тест других моделей

In [57]:
params_choose = {'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.25, 0.5],
                  'max_depth': [1, 2, 3, 4, 5, 6, 7, 8],
                  'n_estimators': [10, 15, 20, 25, 30 ,35, 40, 45, 50, 55 , 60],
                  'seed': [0],
                  'verbosity': [0]}
my_model = XGBClassifier()
search_res = GridSearchCV(my_model, params_choose, scoring=make_scorer(f1_score), cv=GroupKFold(n_splits=5))
search_res.fit(X_train, y_train, groups=train_data['group_id'].values)
best_model = search_res.best_estimator_
search_res.best_params_

{'learning_rate': 0.01,
 'max_depth': 1,
 'n_estimators': 10,
 'seed': 0,
 'verbosity': 0}

In [58]:
cross_val_score(best_model, X_train, y_train, groups=train_data['group_id'].values,
                scoring=make_scorer(f1_score)).mean()

0.7197891267592494

In [59]:
best_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=0)

In [60]:
test_gr = pd.read_csv('test_groups.csv')
y_pred = best_model.predict(X_test)
y_pred = y_pred.astype(int)
result = pd.DataFrame({'pair_id': np.asarray(test_gr['pair_id']), 'target': y_pred})
result = result.set_index(['pair_id'])
result.to_csv('xgb1.csv')